# Big Cat Puzzle

## importing packages

In [110]:
import numpy as np
from gurobipy import *

## Defining constants

In [111]:
values = {
    "TF": 1,
    "LF": 2,
    "PF": 3,
    "JF": 4,
    "TB":-1,
    "LB":-2,
    "PB":-3,
    "JB":-4,
}

translator = {v: k for k, v in values.items()}

TIGER_FRONT = values["TF"]
LION_FRONT = values["LF"]
PHANTER_FRONT = values["PF"]
JAGUAR_FRONT = values["JF"]
TIGER_BACK = values["TB"]
LION_BACK = values["LB"]
PHANTER_BACK = values["PB"]
JAGUAR_BACK = values["JB"]

## Defining Tile class

In [ ]:
class Tile:    
    def __init__(self,left,top,bottom,right,index):
        self.left = left
        self.top = top
        self.bottom = bottom
        self.right = right
        self.index = index
        
    def rotate(self,steps):
        if steps == 0:
            return self
        else:
            return Tile(self.top,self.right,self.left,self.bottom,self.index).rotate(steps - 1)
    def __str__(self):
        return "\n".join([" ".join(e) for e in self.arrstr()])
    
    def arrstr(self):
        return np.array([
            ["  ",translator[self.top],"  "],
            [translator[self.left],"%2.d" % (self.index,),translator[self.right]],
            ["  ",translator[self.bottom],"  "]
        ])

## Defining helper functions

In [ ]:
left = np.vectorize(lambda x : x.left)
top = np.vectorize(lambda x : x.top)
right = np.vectorize(lambda x : x.right)
bottom = np.vectorize(lambda x : x.bottom)

## Initializing tile pieces

In [112]:
basic_tiles = [
    Tile(
        TIGER_FRONT, JAGUAR_BACK,
        LION_FRONT, TIGER_BACK,
        0
    ),
    Tile(
        JAGUAR_BACK,JAGUAR_FRONT,
        PHANTER_FRONT,LION_BACK,
        1
    ),
    Tile(
        LION_BACK,LION_FRONT,
        JAGUAR_FRONT,TIGER_BACK,
        2
    ),
    Tile(
        TIGER_FRONT,TIGER_FRONT,
         LION_BACK,JAGUAR_FRONT,
        3
    ),
    Tile(
        TIGER_BACK, TIGER_BACK,
        PHANTER_FRONT, LION_FRONT,
        4
    ),
    Tile(
        TIGER_FRONT,PHANTER_FRONT,
        JAGUAR_FRONT, LION_BACK,
        5
    ),
    Tile(
        PHANTER_BACK, LION_BACK,
        JAGUAR_FRONT, PHANTER_BACK,
        6
    ),
    Tile(
        JAGUAR_BACK, PHANTER_BACK,
        PHANTER_FRONT, LION_FRONT,
        7
    ),
    Tile(
        TIGER_FRONT, PHANTER_BACK,
        JAGUAR_BACK, PHANTER_FRONT,
        8
    ),
]

for e in basic_tiles:
    print(e)

   JB   
TF  0 TB
   LF   
   JF   
JB  1 LB
   PF   
   LF   
LB  2 TB
   JF   
   TF   
TF  3 JF
   LB   
   TB   
TB  4 LF
   PF   
   PF   
TF  5 LB
   JF   
   LB   
PB  6 PB
   JF   
   PB   
JB  7 LF
   PF   
   PB   
TF  8 PF
   JB   


## Initializing Model

In [113]:
m = Model()

## Adding constrains
### Defining all possible tiles
$i$: index of the tile  
$rot$: rotation of the tile  
$pos$: position of the tile 
  
$tile_{i,rot}$:  defines the value of the tiles.  
$t_{i,rot,pos}$: are the Integer program variables

In [114]:
tiles = np.array([[tile.rotate(rot) for rot in range(4)] for tile in basic_tiles])
tiles_variable = np.array([[[m.addVar(vtype=GRB.BINARY, name="tile_%d_rot_%d_pos_%d" % (tile_index, rot, pos)) for pos in range(9)] for rot in range(4)] for tile_index in range(len(basic_tiles))])

### For each tile, one state only must exist  
$\forall{i} \sum_{rot,pos} t_{i,rot,pos} = 1$

In [115]:
for i, tile_states in enumerate(tiles_variable):
    m.addConstr(np.sum(tile_states) == 1,name="one_state_for_tile_%d" % (i,))

### For each slot, one tile only must exist  
$\forall{pos} \sum_{i,rot} t_{i,rot,pos} = 1$

In [116]:
for pos in range(9):
    m.addConstr(np.sum(tiles_variable[:,:,pos]) == 1, name="one_tile_in_pos_%d" % (pos,))

The slot indexing is the following  
<table>
  <tr>
    <th>0</th>
    <th>1<br></th>
    <th>2</th>
  </tr>
  <tr>
    <td>3</td>
    <td>4</td>
    <td>5</td>
  </tr>
  <tr>
    <td>6</td>
    <td>7</td>
    <td>8</td>
  </tr>
</table>  

### Side matching  
$ \sum_{i,rot}{right(tile_{i,rot}) \cdot t_{i,rot,0}} + \sum_{i,rot}{left(tile_{i,rot}) \cdot t_{i,rot,1}} = 0$  

$ \sum_{i,rot}{right(tile_{i,rot}) \cdot t_{i,rot,1}} + \sum_{i,rot}{left(tile_{i,rot}) \cdot t_{i,rot,2}} = 0$  

$ \sum_{i,rot}{right(tile_{i,rot}) \cdot t_{i,rot,3}} + \sum_{i,rot}{left(tile_{i,rot}) \cdot t_{i,rot,4}} = 0$  

$ \sum_{i,rot}{right(tile_{i,rot}) \cdot t_{i,rot,4}} + \sum_{i,rot}{left(tile_{i,rot}) \cdot t_{i,rot,5}} = 0$  

$ \sum_{i,rot}{right(tile_{i,rot}) \cdot t_{i,rot,6}} + \sum_{i,rot}{left(tile_{i,rot}) \cdot t_{i,rot,7}} = 0$  

$ \sum_{i,rot}{right(tile_{i,rot}) \cdot t_{i,rot,7}} + \sum_{i,rot}{left(tile_{i,rot}) \cdot t_{i,rot,8}} = 0$  
  
### Top and bottom matching
$ \sum_{i,rot}{bottom(tile_{i,rot}) \cdot t_{i,rot,0}} + \sum_{i,rot}{top(tile_{i,rot}) \cdot t_{i,rot,3}} = 0$  

$ \sum_{i,rot}{bottom(tile_{i,rot}) \cdot t_{i,rot,1}} + \sum_{i,rot}{top(tile_{i,rot}) \cdot t_{i,rot,4}} = 0$  

$ \sum_{i,rot}{bottom(tile_{i,rot}) \cdot t_{i,rot,2}} + \sum_{i,rot}{top(tile_{i,rot}) \cdot t_{i,rot,5}} = 0$  

$ \sum_{i,rot}{bottom(tile_{i,rot}) \cdot t_{i,rot,3}} + \sum_{i,rot}{top(tile_{i,rot}) \cdot t_{i,rot,6}} = 0$  

$ \sum_{i,rot}{bottom(tile_{i,rot}) \cdot t_{i,rot,4}} + \sum_{i,rot}{top(tile_{i,rot}) \cdot t_{i,rot,7}} = 0$  

$ \sum_{i,rot}{bottom(tile_{i,rot}) \cdot t_{i,rot,5}} + \sum_{i,rot}{top(tile_{i,rot}) \cdot t_{i,rot,8}} = 0$  

In [117]:
def side_matching(m,tiles,tiles_variable,i,j):
    m.addConstr(np.sum(right(tiles)*tiles_variable[:,:,i]) + np.sum(left(tiles)*tiles_variable[:,:,j]) == 0, name="tile_%d_right_equal_tile_%d_left" % (i,j))
    
def vertical_matching(m,tiles,tiles_variable,i,j):
    m.addConstr(np.sum(bottom(tiles)*tiles_variable[:,:,i]) + np.sum(top(tiles)*tiles_variable[:,:,j]) == 0, name="tile_%d_bottom_equal_tile_%d_top" % (i,j))

side_matching(m,tiles,tiles_variable,0,1)
side_matching(m,tiles,tiles_variable,1,2)
side_matching(m,tiles,tiles_variable,3,4)
side_matching(m,tiles,tiles_variable,4,5)
side_matching(m,tiles,tiles_variable,6,7)
side_matching(m,tiles,tiles_variable,7,8)

vertical_matching(m,tiles,tiles_variable,0,3)
vertical_matching(m,tiles,tiles_variable,1,4)
vertical_matching(m,tiles,tiles_variable,2,5)
vertical_matching(m,tiles,tiles_variable,3,6)
vertical_matching(m,tiles,tiles_variable,4,7)
vertical_matching(m,tiles,tiles_variable,5,8)

In [118]:
m.write("model.lp")
m.optimize()
m.write("model.sol")

Optimize a model with 30 rows, 324 columns and 1512 nonzeros
Variable types: 0 continuous, 324 integer (324 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 30 rows, 324 columns, 1404 nonzeros
Variable types: 0 continuous, 324 integer (324 binary)

Root relaxation: objective 0.000000e+00, 49 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   26          -    0.00000      -     -    0s
     0     0    0.00000    0   32          -    0.00000      -     -    0s
     0     0    0.00000    0   32          -    0.00000      -     -    0s
     0     0    0.00000    0   32          -    0.00000      -     -    0s
     0     2    0.00000    0    8          -    0.00000      -     -    0s
*  4

## Parse solution

In [119]:
output = np.array([["  " for _ in range(9)] for _ in range(9)])

tiles_value = np.vectorize(lambda x : x.X)(tiles_variable)
for tile_index, rot, pos  in np.argwhere(tiles_value):
    output[3*(pos//3):3*(pos//3) + 3,3*(pos%3):3*(pos%3) + 3] = tiles[tile_index,rot].arrstr()

## Solution

In [120]:
print("\n".join([" ".join(e) for e in output]))

   JF       PF       LB   
TF  3 LB LF  7 JB JF  2 LF
   TF       PB       TB   
   TB       PF       TF   
PF  4 TB TF  5 LB LF  0 JB
   LF       JF       TB   
   LB       JB       TF   
PB  6 PB PF  1 JF JB  8 PB
   JF       LB       PF   
